# SMT

In [13]:
from itertools import combinations
from z3 import *

In [14]:
#Caricamento dati

file = open("./Instances/inst08.dat", "r")
lines = file.readlines()
file.close()
m = int(lines[0].rstrip(''))
n = int(lines[1].rstrip(''))
load = [int(x) for x in lines[2].split(' ')]
size = [int(x) for x in lines[3].split(' ')]
instance = [[int(x) for x in line.rstrip().split(' ') ]for line in lines[4:]]
o = n
print(m)
print(n)
print(load)
print(size)
for i in range(n+1):
    for j in range(n+1):
        print(instance[i][j], end = " ")
    print()

#Definizione solver
# s = Solver()
s = Optimize()

8
10
[180, 160, 185, 200, 180, 160, 175, 200]
[10, 25, 18, 16, 14, 7, 9, 16, 24, 19]
0 56 86 87 81 128 107 163 166 98 93 
56 0 80 31 63 87 61 107 119 76 37 
86 71 0 102 17 42 111 77 90 12 70 
87 31 110 0 93 116 30 77 89 105 40 
81 63 17 93 0 47 94 82 94 17 53 
128 87 42 116 47 0 117 52 92 30 76 
117 61 111 30 94 117 0 65 59 106 41 
163 107 77 77 82 52 65 0 40 65 70 
166 110 90 79 94 92 49 40 0 82 82 
98 76 12 105 17 30 106 65 82 0 65 
93 37 70 40 53 76 41 70 82 65 0 


In [ ]:
#Utils
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars))

def all_false(bool_vars):
    return And([And(Not(pair[0]), Not(pair[1])) for pair in combinations(bool_vars, 2)])

: 

In [16]:
#Decision variables
assignment = [[Bool(f'assignment_{i}_{j}') for j in range(m)] for i in range(n)]

# tour[i][j] = x ---> courier i dispatch item x at j-th route
tour = [[Int(f'tour_{i}_{j}') for j in range(n+2)] for i in range(m)]

# Distance matrix
# D = [[Int(f'D_{i}_{j}') for j in range(n+1)] for i in range(n+1)]
D_val = ArraySort(IntSort(), IntSort())
D = Array('D', IntSort(), D_val)

# Maxdist objective variable
maxDist = Int("maxDist")

In [17]:
# Constraints

# Matrix D initialization
for i in range(n+1):
    for j in range(n+1):
        s.add(D[i][j] == instance[i][j])

# Each item is assigned to exactly one courier
for i in range(n):
    s.add(exactly_one_np([assignment[i][j] for j in range(m)]))

# Each courier cannot load more than his capacity
for j in range(m):
    s.add(sum([If(assignment[i][j], size[i], 0) for i in range(n)]) <= load[j])

# Initial and final destination are the same
for c in range(m):
    s.add(And(tour[c][0] == o, tour[c][n+1] == o))

# Assigned items must be in courier's tour
for c in range(m):
    for i in range(n):
        s.add(Implies(assignment[i][c], at_least_one_np([tour[c][j] == i for j in range(1, n+1)])))

# Each assigned item is dispatched just once
for c in range(m):
    for i in range(n):
        s.add(Implies(assignment[i][c], exactly_one_np([tour[c][j] == i for j in range(1,n+1)])))

# If courier c does not dispatch item i, then i is not in his tour
for c in range(m):
    for i in range(n):
        s.add(Implies(Not(assignment[i][c]), all_false([tour[c][j] == i for j in range(1, n+1)])))

# Can't go back to deposit
for c in range(m):
    for j in range(1,n):
        s.add(Implies(tour[c][j] == o, tour[c][j+1] == o))

# Having an item in the tour implies that the item is assigned
for c in range(m):
    for j in range(1,n+1):
        s.add([Implies(tour[c][j] == i, assignment[i][c]) for i in range(n)])

# Limit the range of assignable item to only valid one
for c in range(m):
    for j in range(1,n+1):
        s.add(And(Not(tour[c][j]>n), Not(tour[c][j]<0)))

In [18]:
# Objective function, minize max distance
maxList = [Sum([D[tour[c][i]][tour[c][i+1]] for i in range(n+1)]) for c in range(m)]

for c in range(m):
    s.add(maxDist >= maxList[c])
z = s.minimize(maxDist)

In [19]:
if s.check() == sat:
    model = s.model()
    print("sat")
    for i in range(n):
        for j in range(m):
            print(model[assignment[i][j]], end = " ")
        print()
    for i in range(m):
        for j in range(n+2):
            print(model[tour[i][j]], end = " ")
        print()
    
    dist = [sum([ instance[int(str(model[tour[i][j]]))][int(str(model[tour[i][j+1]]))] for j in range(n+1) if i in range(n)]) for i in range(m)]
    print("Distances:", dist)
    print("MaxDist:", z.value())
    print("Total Dist:", sum(dist))
else:
    print("unsat")

sat
False True False False False False False False 
False False False False False False False True 
False False False False False False True False 
False False False False False False False True 
False False False False False False True False 
False False False False False True False False 
False False False True False False False False 
False False False False True False False False 
False False False True False False False False 
False False False False False False True False 
10 10 10 10 10 10 10 10 10 10 10 10 
10 0 10 10 10 10 10 10 10 10 10 10 
10 10 10 10 10 10 10 10 10 10 10 10 
10 8 6 10 10 10 10 10 10 10 10 10 
10 7 10 10 10 10 10 10 10 10 10 10 
10 5 10 10 10 10 10 10 10 10 10 10 
10 9 4 2 10 10 10 10 10 10 10 10 
10 3 1 10 10 10 10 10 10 10 10 10 
Distances: [0, 186, 0, 172, 140, 152, 169, 108]
MaxDist: 186
Total Dist: 927
